# Interactive visualizer
Using [Interactive visualizers](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html) you can bring your own renderer and connect it to the visualizer with live mouse camera control right in the notebook, for example to debug your custom rendering function. The main condition is that the renderer has to take a [Camera](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.camera.camera.html#kaolin-render-camera-camera) as input.

In this notebook, we show how to visualize a mesh from ShapeNet using spherical gaussians lighting and our new (since v0.16.0) `easy_render` differentiable render (you can plug in your own render function instead). 

In [1]:
import copy
import glob
import math
import logging
import numpy as np
import os
import sys
import torch

from tutorial_common import COMMON_DATA_DIR
import kaolin as kal

def print_tensor(t, *args, **kwargs):
    print(kal.utils.testing.tensor_info(t, *args, **kwargs))

## Load Mesh information

In [2]:
# Set KAOLIN_TEST_SHAPENETV2_PATH env variable, or replace by your shapenet path
SHAPENETV2_PATH = os.getenv('KAOLIN_TEST_SHAPENETV2_PATH')

if SHAPENETV2_PATH is not None:
    ds = kal.io.shapenet.ShapeNetV2(root=SHAPENETV2_PATH,
                                    categories=['car'],
                                    train=True, split=1.,
                                    with_materials=True,
                                    output_dict=True)
    mesh = ds[0]['mesh']
    mesh.materials = [kal.io.obj.raw_material_to_pbr(m) for m in mesh.materials]  # convert to PBR material
else:
    # Load a specific obj instead
    OBJ_PATH = os.path.join(COMMON_DATA_DIR, 'meshes', 'fox.obj')
    mesh = kal.io.obj.import_mesh(OBJ_PATH, with_materials=True, with_normals=True, triangulate=True,
                                  raw_materials=False)  # convert to PBR material

def process_mesh(mesh):
    if mesh.materials is not None and len(mesh.materials) > 0:
        print(str(mesh.materials[0]))
        
    # Batch, move to GPU and center and normalize vertices in the range [-0.5, 0.5]
    mesh = mesh.to_batched().cuda()
    mesh.vertices = kal.ops.pointcloud.center_points(mesh.vertices, normalize=True)
    print(mesh)
    return mesh

mesh = process_mesh(mesh)

PBRMaterial object with
                    material_name: material_0
                      shader_name: UsdPreviewSurface
                    diffuse_color: [3] (torch.float32)[cpu]   tensor([0.8000, 0.8000, 0.8000])
                   specular_color: [3] (torch.float32)[cpu]   tensor([1., 1., 1.])
                  diffuse_texture: [1024, 1024, 3] (torch.float32)[cpu]  
SurfaceMesh object with batching strategy FIXED
            vertices: [1, 5002, 3] (torch.float32)[cuda:0]  
               faces: [10000, 3] (torch.int64)[cuda:0]  
             normals: [1, 5002, 3] (torch.float32)[cuda:0]  
    face_normals_idx: [1, 10000, 3] (torch.int64)[cuda:0]  
                 uvs: [1, 5505, 2] (torch.float32)[cuda:0]  
        face_uvs_idx: [1, 10000, 3] (torch.int64)[cuda:0]  
material_assignments: [1, 10000] (torch.int16)[cuda:0]  
           materials: [
                      0: list of length 1
                      ]
       face_vertices: if possible, computed on access from: (faces, ve

## Instantiate a camera

With the general constructor `Camera.from_args()` the underlying constructors are `CameraExtrinsics.from_lookat()` and `PinholeIntrinsics.from_fov` we will use this camera as a starting point for the visualizers.

In [3]:
camera = kal.render.camera.Camera.from_args(eye=torch.tensor([2., 1., 1.], device='cuda'),
                                            at=torch.tensor([0., 0., 0.]),
                                            up=torch.tensor([1., 1., 1.]),
                                            fov=math.pi * 45 / 180,
                                            width=512, height=512, device='cuda')

## Rendering a mesh

Here we are rendering the loaded mesh with [nvdiffrast](https://github.com/NVlabs/nvdiffrast) using the camera object created above and use both diffuse and specular reflectance for lighting.

For more information on lighting in Kaolin see [diffuse](./diffuse_lighting.ipynb) and [specular](./sg_specular_lighting.ipynb) tutorials and the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.lighting.html).

In [4]:
# Those are the parameters used to define the Spherical gaussian
apply_lighting = True
azimuth = torch.zeros((1,), device='cuda')
elevation = torch.full((1,), math.pi / 3., device='cuda')
amplitude = torch.full((1, 3), 3., device='cuda')
sharpness = torch.full((1,), 5., device='cuda')

def current_lighting():
    direction = kal.render.lighting.sg_direction_from_azimuth_elevation(azimuth, elevation)
    return kal.render.lighting.SgLightingParameters(
        amplitude=amplitude, sharpness=sharpness, direction=direction)


def base_render(mesh, camera, lighting, clear=False, active_pass=kal.render.easy_render.RenderPass.render):
    render_res = kal.render.easy_render.render_mesh(camera, mesh, lighting=lighting)
    
    img = render_res[active_pass]
    hard_mask = (render_res[kal.render.easy_render.RenderPass.face_idx] >= 0).float().unsqueeze(-1)
    
    if clear:
        img = torch.cat([img, hard_mask], dim=-1)  # Add Alpha channel
    final = (torch.clamp(img * hard_mask, 0., 1.)[0] * 255.).to(torch.uint8)

    return final

def make_lowres_cam(in_cam, factor=8):
    lowres_cam = copy.deepcopy(in_cam)
    lowres_cam.width = in_cam.width // 8
    lowres_cam.height = in_cam.height // 8
    return lowres_cam

def render(in_cam):
    active_pass = kal.render.easy_render.RenderPass.render if apply_lighting else kal.render.easy_render.RenderPass.albedo
    return base_render(mesh, in_cam, current_lighting(), active_pass=active_pass)

def make_lowres_render_func(render_func):
    def lowres_render_func(in_cam):
        return render_func(make_lowres_cam(in_cam))
    return lowres_render_func

## Turntable visualizer
This is a simple visualizer useful to inspect a small object.

You can move around with the mouse (left button) and zoom with the mouse wheel.
See the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html#kaolin.visualize.IpyTurntableVisualizer) to customize the sensitivity.

In [5]:
visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render, fast_render=make_lowres_render_func(render),
    max_fps=24, world_up_axis=1)
visualizer.show()

Canvas(height=512, width=512)

Output()

## First person visualizer
This is a visualizer useful to inspect details on an object, or a big scene.

You can move the orientation of the camera with the mouse left button, move the camera around with the mouse right button or
the keys 'i' (up), 'k' (down), 'j' (left), 'l' (right), 'o' (forward), 'u' (backward)

See the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html#kaolin.visualize.IpyFirstPersonVisualizer) to customize the sensitivity and keys.

--------------------
*Note: camera are mutable in the visualizer. If you want to keep track of the camera position you can remove the `copy.deepcopy` on camera argument or you can check `visualizer.camera`*

In [6]:
visualizer = kal.visualize.IpyFirstPersonVisualizer(
    512, 512, copy.deepcopy(camera), render, fast_render=make_lowres_render_func(render),
    max_fps=24, world_up=torch.tensor([0., 1., 0.], device='cuda'))
visualizer.show()

Canvas(height=512, width=512)

Output()

## Adding events and other widgets

The visualizer is modular.
Here we will add:
* sliders to control the spherical gaussian parameters (see [ipywidgets tutorial](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html) for more info).
* A key event to 'space' to enable / disable specular reflectance (see [ipyevents documentation](https://github.com/mwcraig/ipyevents/blob/main/docs/events.ipynb)) to see all the events that can be caught.

In general if you want to modify the rendering function you can use global variables or make a class (with the rendering function being a method)

-------------
More info on spherical gaussians parameters in our [sg_specular_lighting.ipynb](./sg_specular_lighting.ipynb) tutorial
and [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.lighting.html).

In [8]:
from ipywidgets import interactive, HBox, FloatSlider, VBox, Dropdown

def additional_event_handler(visualizer, event):
    """Event handler to be provided to Kaolin's visualizer"""
    with visualizer.out: # This is for catching print and errors
        if event['type'] == 'keydown' and event['key'] == ' ':
            global apply_lighting
            apply_lighting = not apply_lighting
            visualizer.render_update()
            return False
        return True

visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render, fast_render=make_lowres_render_func(render),
    max_fps=24,
    additional_event_handler=additional_event_handler,
    additional_watched_events=['keydown'] # We need to now watch for key press event
)
# we don't call visualizer.show() here

def sliders_callback(new_elevation, new_azimuth, new_amplitude, new_sharpness):
    """ipywidgets sliders callback"""
    with visualizer.out: # This is in case of bug
        elevation[:] = new_elevation
        azimuth[:] = new_azimuth
        amplitude[:] = new_amplitude
        sharpness[:] = new_sharpness
        # this is how we request a new update
        visualizer.render_update()
        
elevation_slider = FloatSlider(
    value=elevation.item(),
    min=-math.pi / 2.,
    max=math.pi / 2.,
    step=0.1,
    description='Elevation:',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

azimuth_slider = FloatSlider(
    value=azimuth.item(),
    min=-math.pi,
    max=math.pi,
    step=0.1,
    description='Azimuth:',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

amplitude_slider = FloatSlider(
    value=amplitude[0,0].item(),
    min=0.1,
    max=20.,
    step=0.1,
    description='Amplitude:\n',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

sharpness_slider = FloatSlider(
    value=sharpness.item(),
    min=0.1,
    max=20.,
    step=0.1,
    description='Sharpness:\n',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

interactive_slider = interactive(
    sliders_callback,
    new_elevation=elevation_slider,
    new_azimuth=azimuth_slider,
    new_amplitude=amplitude_slider,
    new_sharpness=sharpness_slider
)

# We combine all the widgets and the visualizer canvas and output in a single display
full_output = HBox([visualizer.canvas, interactive_slider])
display(full_output, visualizer.out)

Output()

## Customizing Drawing and Event Canvases

In some cases, it may be desirable to receive events on a different canvas from the one used for drawing, or you may want to create the drawing canvas manually. 

You can mix and match any number of drawing canvasses and event canvasses by passing `canvas` and `event_canvas` variables to the visualizer constructor. 

In this example, we show:

1. How to use Kaolin visualizers with MultiCanvas, where you may draw something else on one of the stacked sub-canvasses (in this case, background color)
2. How to control visualizer using events from another canvas (in this case, mouse motion in the first canvas controls both cameras)

There are many ways you may want to build debug interfaces around your rendering function, and our visualizers allow for flexibility.

In [9]:
from ipycanvas import MultiCanvas, Canvas, hold_canvas
from ipywidgets import Layout, VBox, Dropdown
from functools import partial

# Create two multi-layer canvasses, containing 2 aligned images each with opacity enabled
# (e.g. useful for various overlays)
# multi_canvas[0] - background canvas
# multi_canvas[1] - foreground canvas
cwidth = 512
multi_canvas = MultiCanvas(2, width=cwidth, height=cwidth, layout=Layout(width="500px", height="500px"))
multi_canvas2 = MultiCanvas(2, width=cwidth, height=cwidth, layout=Layout(width="500px", height="500px"))
colors = ["red", "blue", "green", "purple", "#cc3333"]

# Function to set background canvas of both canvasses
def set_background(val): 
    with hold_canvas():
        multi_canvas[0].fill_style = val
        multi_canvas[0].fill_rect(0, 0, cwidth, cwidth)
        multi_canvas2[0].fill_style = val
        multi_canvas2[0].fill_rect(0, 0, cwidth, cwidth)
        
# Actually set background and clear front canvasses
set_background(colors[0])
with hold_canvas():
    multi_canvas[1].clear_rect(0, 0, cwidth, cwidth)
    multi_canvas2[1].clear_rect(0, 0, cwidth, cwidth)
    
# Create background color picker
def handle_dropdown(change):
    global background_color
    with visualizer.out:
        set_background(change['new'])
color_dropdown = Dropdown(options=colors, value='red', description='Background:')
color_dropdown.observe(handle_dropdown, names='value')


# Read in an additional mesh
mesh2 = kal.io.obj.import_mesh(os.path.join(COMMON_DATA_DIR, 'meshes', 'pizza.obj'), with_materials=True, with_normals=True, triangulate=True, raw_materials=False)
mesh2 = process_mesh(mesh2)
    
# Create render closures that only requires a camera to render; ensure we render with Alpha channel
lighting = current_lighting()
render1 = partial(base_render, mesh, lighting=lighting, clear=True)
render2 = partial(base_render, mesh2, lighting=lighting,  clear=True)

# Create first visualizer
visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render1, fast_render=make_lowres_render_func(render1),
    max_fps=24, world_up_axis=1,
    canvas=multi_canvas[1], event_canvas=multi_canvas)

# Create second visualizer, still controlled by mouse within the first canvas
visualizer2 = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render2, fast_render=make_lowres_render_func(render2),
    max_fps=24, world_up_axis=1,
    canvas=multi_canvas2[1], event_canvas=multi_canvas)

# Show all the canvasses and outputs
visualizer.render_update()
visualizer2.render_update()
VBox((HBox((multi_canvas,multi_canvas2)), color_dropdown, HBox((visualizer.out, visualizer2.out))))

PBRMaterial object with
                    material_name: dough
                      shader_name: UsdPreviewSurface
                    diffuse_color: [3] (torch.float32)[cpu]   tensor([0.8000, 0.5487, 0.2568])
                   specular_color: [3] (torch.float32)[cpu]   tensor([0.1519, 0.1519, 0.1519])
SurfaceMesh object with batching strategy FIXED
            vertices: [1, 482, 3] (torch.float32)[cuda:0]  
               faces: [960, 3] (torch.int64)[cuda:0]  
             normals: [1, 482, 3] (torch.float32)[cuda:0]  
    face_normals_idx: [1, 960, 3] (torch.int64)[cuda:0]  
                 uvs: [1, 514, 2] (torch.float32)[cuda:0]  
        face_uvs_idx: [1, 960, 3] (torch.int64)[cuda:0]  
material_assignments: [1, 960] (torch.int16)[cuda:0]  
           materials: [
                      0: list of length 2
                      ]
       face_vertices: if possible, computed on access from: (faces, vertices)
        face_normals: if possible, computed on access from: (normals, 